# Regular Expression

![title](https://imgs.xkcd.com/comics/regular_expressions.png)

A **regular expression** (RE) is a compact notation for representing a collection of strings.
REs are defined using a mini-language different from Python and are used to search and substitute patterns and to parse, split, and validate strings.

In Python, REs and REs functionalities are defined in the `re` module.

In [1]:
import re

The most common task with REs is to search a given pattern in a string.
Let's start with searching a single character in a string.

In [2]:
occurrence = re.search(pattern='u', string='George Lucas')

`re.search` returns a [Match object](https://docs.python.org/3/library/re.html#match-objects).

In [3]:
type(occurrence)

_sre.SRE_Match

The Match object evaluate to True if the pattern appears in the string, otherwise it is None.

In [4]:
z_occurrence = re.search('z', 'George Lucas')

if occurrence:
    print("occurrence evalutates to True.")
else:
    print("occurrence does not evaluate to True.")
    
if z_occurrence:
    print('z_occurrence evaluates to True.')
else:
    print('z_occurrence does not evaluate to True.')

occurrence evalutates to True.
z_occurrence does not evaluate to True.


The match object can be queried to retrieve the information about the match occurrence.

In [5]:
print("Pattern searched: {}\nString Provided: {}".format(occurrence.re.pattern,
                                                         occurrence.string))
print("Occurrence start: {}\nOccurrence end: {}\nMatch interval: {}\nString: {}".format(occurrence.start(),
                                                                                        occurrence.end(),
                                                                                        occurrence.span(),
                                                                                        occurrence.group()))

Pattern searched: u
String Provided: George Lucas
Occurrence start: 8
Occurrence end: 9
Match interval: (8, 9)
String: u


## Basics of the RE langage

In a RE every character matches itself except for the following characters.

| Character | Meaning |
|:--- | :--- |
| ^ | Beginning of line |
| $ | End of line |
| * | Match 0+ times the previous character in RE |
| + | Match 1+ times the previous character in RE |
| ? | Match 0 or 1 times the previous character in RE |

It is possible to match any of the character in the previous table by "escaping" it using `\`.  For example, `\+` matches the character `+`.
To group multiple characters just write them the correct sequence.

Moreover, characters `[` and `]` are used to group together other characters in a character class.
For example `[abcd]` means *"any character between a, b, c and d"*.
`[abcd]` and `[a-d]` are the same character class.

To match multiple patterns, group all the patterns between parentheses, and divide them using `|`.
For example `(ab|ca|de)` matches `ab`, `ca`, and `de`.

If `^` is inside a character class, it complements the elements inside the class.  This means that `[^a-d]` represents any character that is not a, b, c, nor d.

For example, given the lyrics of this famous song

In [6]:
lyrics="""La, la, la, la, la, la, la, la
La, la, la, la, la, la, la, la
La, la, la, la, la, la, la, la
La, la, la, la, la, la, la, la
I just can't get you out of my head
Boy, your lovin' is all I think about
I just can't get you out of my head
Boy, its more than I dare to think about
La, la, la, la, la, la, la, la
La, la, la, la, la, la, la, la
I just can't get you out of my head
Boy, your lovin' is all I think about
I just can't get you out of my head
Boy, its more than I dare to think about
Every night
Every day
Just to be there in your arms
Won't you stay
Won't you stay
Stay forever and ever and ever ah ah
La, la, la, la, la, la, la, la
La, la, la, la, la, la, la, la
La, la, la, la, la, la, la, la
La, la, la, la, la, la, la, la
I just can't get you out of my head
Boy, your lovin' is all I think about
I just can't get you out of my head
Boy, its more than I dare to think about
There's a dark secret in me
Don't leave me lost in your arms
Set me free
Feel the need in me
Set me free
Stay forever and ever and ever ah ah
La, la, la, la, la, la, la, la
La, la, la, la, la, la, la, la
La, la, la, la, la, la, la, la
La, la, la, la, la, la, la, la
I just can't get you out of my head (La, la, la La, la, la, la, la)""".split('\n')

If we want do something on all the verses but the chorus verses ("La, la, la, la [...]") we can use the following code snippets.

In [7]:
def do_something(line):
    # Do whatever, do nothing now
    # print(line)
    return line

In [8]:
%%timeit
for line in lyrics:
    occ = re.search('^(La, |la, )+la$', line)
    if not occ:
        do_something(line)

The slowest run took 9.74 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 35.8 µs per loop


In [9]:
%%timeit
for line in lyrics:
    occ = re.search('^([Ll]a, )+la$', line)
    if not occ:
        do_something(line)

The slowest run took 5.83 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 34 µs per loop


## Character Class Shorthands

Python also defines some metacharacters for common character classes.

| Character | Meaning |
|:--- | :--- |
| `.` | Any character |
| `\d` | Any digit, equivalent to `[0-9]` |
| `\D` | Any nondigit, equivalent to `[^0-9] |
| `\s` | Any whitespace (`\t`, `\n`, ...) |
| `\S` | Any nonwhitespace |
| `\w` | Any word character, equivalent to `[a-zA-Z0-9_]` |
| `\W` | Any nonword character |

## Quantifiers

A quantifier appears after an expression of a RE and has the form `{m, n}` where `m` and `n` are the minimum and maximum times the expression the quantifier applies to must match.

For example
```python
re.search('(bla){2,100}', 'blablabla')
```
will return a match whereas
```python
re.search('(bla){4,100}', 'blablabla')
```
won't.

In [10]:
m = re.search('(bla){2,100}', 'blablabla')
print(m)

<_sre.SRE_Match object; span=(0, 9), match='blablabla'>


In [11]:
m = re.search('(bla){4,100}', 'blablabla')
print(m)

None


## Compiling REs

Usually REs are applied over and over.
To speed up the processing it is possible to *compile* the regular expression into a compiled regex object and use it as many times as we like.
For example, we can rewrite one of the previous examples as follows.

In [12]:
%%timeit
la_re = re.compile('^(La, |la, )+la$')
for line in lyrics:
    occ = la_re.search(line)
    if not occ:
        do_something(line)

10000 loops, best of 3: 24.1 µs per loop


## Compilation flags

Compilation flags let you modify some aspects of how REs work and are passed as arguments to the `re.compile` function.
The most important compilation flag for us will be `re.IGNORECASE` (also `re.I`) that enables to match RE in a case insensitive fashion.
This means that a lowercase and uppercase character will be treated as the same element (i.e., there will be no distinction between `a` and `A`).
For a more in-depth analysis of compilation flags please refer to the [documentation](https://docs.python.org/3.6/howto/regex.html#compilation-flags).

In [13]:
%%time
la_re = re.compile('^(la, )+la$', re.IGNORECASE)
for line in lyrics:
    occ = la_re.search(line)
    if not occ:
        do_something(line)

CPU times: user 403 µs, sys: 6 µs, total: 409 µs
Wall time: 417 µs


# RE functions

For now we only used the `re.search` function but there are many more.
The following table summarizes the most important functions.

| Function | Description |
|:--- |:--- |
| [`re.findall(string)`](https://docs.python.org/3/library/re.html#re.Pattern.findall) | Returns all nonoverlapping matches of the RE (as a list of strings)|
| [`re.match(string)`](https://docs.python.org/3/library/re.html#re.Pattern.match) | Returns a match object if the RE matches *at the start* of the string |
| [`re.search(string)`](https://docs.python.org/3/library/re.html#re.Pattern.search) | Returns a match object if the RE matches anywhere in the string |
| [`re.split(string)`](https://docs.python.org/3/library/re.html#re.Pattern.split) | Returns the list of strings that results from splitting string s on every occurrence of the RE |
| [`re.sub(repl, string)`](https://docs.python.org/3/library/re.html#re.Pattern.sub) | Returns a coput of the string with every match of the RE replaced by `repl` |

## Grouping

Frequently you need to obtain more information than just whether the RE matched or not. Regular expressions are often used to dissect strings by writing a RE divided into several subgroups which match different components of interest [(see this linki)](https://docs.python.org/3/howto/regex.html#grouping).
You can access the groups by using the `group()` method of the match.

For example

In [14]:
name_surname_re = re.compile("^(\d+)\s+(\w+)\s+(\w+)$", re.IGNORECASE)

m = name_surname_re.search("3          Giorgio   Chiellini")
print("Whole match:  ", m.group())
print("Name:         ", m.group(2))
print("Surname:      ", m.group(3))

m = name_surname_re.search("14 Blaise                        Matuidi")
print("Whole match:  ", m.group())
print("Shirt number: ", m.group(1))
print("Surname:      ", m.group(3))

Whole match:   3          Giorgio   Chiellini
Name:          Giorgio
Surname:       Chiellini
Whole match:   14 Blaise                        Matuidi
Shirt number:  14
Surname:       Matuidi


## Named groups

Accessing groups by their position in the match is most of the times tedious.
Python introduces **named group** to refer to each group using a key.
The syntax for a named group is `(?P<key>*expression*)` were `<key>` is the key used and `*expression*` is the RE we're looking for.

In [15]:
ns_named_re = re.compile("^(?P<shirtnumber>\d+)\s+(?P<name>\w+)\s+(?P<surname>\w+)$")

m = ns_named_re.search("3 Giorgio            Chiellini")
print("Whole match:  ", m.group())
print("Name:         ", m.group("name"))
print("Surname:      ", m.group("surname"))

m = ns_named_re.search("14 Blaise                        Matuidi")
print("Whole match:  ", m.group())
print("Shirt number: ", m.group("shirtnumber"))
print("Surname:      ", m.group("surname"))

Whole match:   3 Giorgio            Chiellini
Name:          Giorgio
Surname:       Chiellini
Whole match:   14 Blaise                        Matuidi
Shirt number:  14
Surname:       Matuidi


## Backreferences

Backreferences in a pattern allow you to specify that the contents of an earlier capturing group must also be found at the current location in the string. For example, `\1` will succeed if the exact contents of group 1 can be found at the current position, and fails otherwise.

Backreferences are usueful in many operations; for example if we want to detect doubled words in a string the following RE will suffice.

In [16]:
doubled_words = re.findall(r'\b(\w+)\s+\1\b',
                          'writing doubled doubled words is is a common mistake while writing papers and reports reports')
print(doubled_words)

['doubled', 'is', 'reports']


We can also use backreferences to remove doubled words from a sentence.

In [17]:
doubled_words_removed = re.sub(r'\b(\w+)\s+\1\b', r'\1',
                              'writing doubled doubled words is is a common mistake while writing papers and reports reports')

print(doubled_words_removed)

writing doubled words is a common mistake while writing papers and reports


Note that this RE only works for non-overlapping matches!

In [18]:
doubled_words_removed = re.sub(r'\b(\w+)\s+\1\b', r'\1',
                              'three three three')

print(doubled_words_removed)

three three


# Exercises




**1)** The file `ex-data/numbers.txt` contains 10000 lines.  Each line contains either a number or a string, find how many even number are in the file.

In [19]:
is_even_re = re.compile('[0-9]*[02468]$')
even_numbers = 0

with open('ex-data/numbers.txt') as fin:
    for line in fin:
        line = line.strip('\n')
        if is_even_re.search(line):
            even_numbers += 1

print(even_numbers)

4789


**2)** The file `ex-data/email.txt` contains 80000 lines.
Each line might start with some whitespaces (>=0), is followed by an email address, a number (>0) of spaces, and an age.
Each email address is composed as follows:
* a name
* a separation character (either `.`, `_`, or `!`)
* a surname
* an optional integer number
* a @ symbol
* a domain

Using REs and the concepts from previous lessons, find how many times each domain was used.

Examples of possible lines in the file:
```
riva!menist57@bofthew.com   70  
Brittan_Knorr69@tyldd.com            42  
Cammy_Shawcroft@antichef.net           28  
Lizzie!woolford@dispostable.com 75  
Lurette_beachel@fakeinbox.com           48  
moria.ivery54@twinmail.de          64  
luciana.Leclaire48@gowikibooks.com       50  
```   

In [20]:
email_named_re = re.compile("^\s*(?P<name>[a-zA-Z\-]+)[\._!](?P<surname>[a-zA-Z]+)(?P<intnum>\d*)@(?P<domain>[\w\.\-_]+)\s+\d+$")

domains = {}

fin = open("ex-data/email.txt", 'r')
for line in fin:
    line = line.strip('\n')
    m = email_named_re.match(line)
    d = m.group('domain')
    if d in domains:
        domains[d] += 1
    else:
        domains[d] = 1
fin.close()

for d in domains:
    print("{} / {}".format(d, domains[d]))

fastkawasaki.com / 170
www.mailinator.com / 169
qq.com / 151
antispam.de / 175
pjjkp.com / 154
jnxjn.com / 160
rejectmail.com / 176
cubiclink.com / 171
imails.info / 179
mailslite.com / 172
twinmail.de / 179
675hosting.org / 168
mypartyclip.de / 168
spamcero.com / 144
spambob.net / 174
dumpyemail.com / 170
trashymail.com / 184
upliftnow.com / 169
mx0.wwwnew.eu / 172
trash-amil.com / 189
netzidiot.de / 165
hochsitze.com / 164
thankyou2010.com / 173
fr33mail.info / 169
zippymail.info / 178
nus.edu.sg / 146
trashymail.net / 200
rtrtr.com / 163
a-bc.net / 176
myspamless.com / 169
emailmiser.com / 171
poofy.org / 163
m4ilweb.info / 174
moncourrier.fr.nf / 162
mailmetrash.com / 171
viewcastmedia.org / 178
politikerclub.de / 179
gishpuppy.com / 191
mintemail.com / 136
cosmorph.com / 165
guerillamail.org / 158
yopmail.net / 180
mailzilla.org / 171
dodgit.com / 187
despammed.com / 161
klassmaster.net / 153
0815.ru / 177
spamavert.com / 169
dm.w3internet.co.ukexample.com / 176
nospamthanks.info 

**3)** The file `ex-data/exp_nums.txt` contains 100 lines.  Each line contains a number in [E-notation](https://en.wikipedia.org/wiki/Scientific_notation#E-notation).  Convert each number to its decimal representation.

In [21]:
exp_num_named_re = re.compile("^(?P<number>\d+.\d+)e(?P<exp>-*\d+)$")

fin = open("ex-data/exp_nums.txt", 'r')
for line in fin:
    line = line.strip('\n')
    m = exp_num_named_re.match(line)
    n = float(m.group('number'))
    e = int(m.group('exp'))
    print("Exp: {:>12} \t Dec: {:>20}".format(line, n*(10**e)))
fin.close()

Exp:    2.19925e5 	 Dec:   219925.00000000003
Exp:    2.98685e3 	 Dec:              2986.85
Exp:    1.40469e2 	 Dec:              140.469
Exp:    2.75051e7 	 Dec:   27505099.999999996
Exp:    3.37623e3 	 Dec:              3376.23
Exp:    2.62878e4 	 Dec:              26287.8
Exp:    1.24722e2 	 Dec:              124.722
Exp:    3.69233e4 	 Dec:              36923.3
Exp:     2.5466e2 	 Dec:   254.66000000000003
Exp:    3.77231e2 	 Dec:              377.231
Exp:    1.55855e4 	 Dec:   15585.500000000002
Exp:    2.64698e3 	 Dec:              2646.98
Exp:    3.19381e3 	 Dec:              3193.81
Exp:    2.77834e6 	 Dec:            2778340.0
Exp:    3.86588e7 	 Dec:           38658800.0
Exp:   3.80454e-1 	 Dec:             0.380454
Exp:   3.34377e-1 	 Dec:  0.33437700000000004
Exp:    2.28575e7 	 Dec:           22857500.0
Exp:    2.17343e6 	 Dec:            2173430.0
Exp:    1.53285e7 	 Dec:           15328500.0
Exp:    1.69785e6 	 Dec:            1697850.0
Exp:    2.38438e6 	 Dec:          

4) [CamelCase](https://en.wikipedia.org/wiki/Camel_case) and [snake_case](https://en.wikipedia.org/wiki/Snake_case) two different ways to name variables.  Write a function (`camel_to_snake`) that converts a string in CamelCase to snake_case.

In [22]:
def camel_to_snake(camelString):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', camelString)
    snake_string = re.sub('([a-z])([A-Z]+)', r'\1_\2', s1)
    return snake_string

In [23]:
print(camel_to_snake('CamelCamelCamel'))
# Must print camel_camel_camel

Camel_Camel_Camel
